In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(100_000_000_000)

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca

import shared_utils
import utils

from siuba import *
import pandas as pd
import geopandas as gpd
import shapely

import datetime as dt
import time
from zoneinfo import ZoneInfo

import rt_analysis as rt
import importlib

importlib.reload(utils)
importlib.reload(rt)

import gcsfs
fs = gcsfs.GCSFileSystem()

/Users/edasmalchi/miniconda3/envs/calitp/lib/python3.10/site-packages/google/cloud/bigquery/client.py:535: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  warnings.warn(


### Generating Analysis from RT Data

* Currently supporting analysis for an entire day from a single operator at a time

In [2]:
rt.OperatorDayAnalysis?

Init signature: rt.OperatorDayAnalysis(itp_id, analysis_date)
Docstring:     
New top-level class for rt delay/speed analysis of a single operator on a single day
    
File:           ~/calitp/data-analyses/rt_delay/rt_analysis2.py
Type:           type
Subclasses:     


In [3]:
analysis_date = dt.date(2022, 1, 12)

In [1]:
# lbt = rt.OperatorDayAnalysis(170, analysis_date)

In [2]:
# marin = rt.OperatorDayAnalysis(194, analysis_date)

### Working from a Single Trip

* Each analysis has a position_interpolators attribute, a nested dict of trip_ids with an rt (and eventually a schedule) interpolator for each
* rt interpolator is the foundation of RT analysis, based on Vehicle Positions
    * trip information
    * ability to interpolate arrival time at arbitrary distances along the route (shape)
    * can generate a detailled single-trip speed map with full resolution from Vehicle Positions

In [29]:
lbt.position_interpolators['9288261']['rt'].time_at_position(5000) ## distance down the shape in meters

Timestamp('2022-01-12 13:47:39.950962')

In [30]:
lbt.position_interpolators['9288261']['rt'].route_id

'192'

In [31]:
lbt.position_interpolators['9288261']['rt'].trip_id

'9288261'

In [21]:
lbt.position_interpolators['9288261']['rt'].position_gdf >> head(3)

,vehicle_timestamp,vehicle_position_latitude,vehicle_position_longitude,header_timestamp,geometry,shape_meters,last_time,last_loc,secs_from_last,meters_from_last,progressed,speed_from_last
0,2022-01-12 13:30:13,33.768063,-118.192120,2022-01-12 13:30:33,POINT (167501.086 -470293.515),44.451926,NaT,NaN,NaN,NaN,False,NaN
1,2022-01-12 13:31:14,33.768085,-118.192210,2022-01-12 13:31:33,POINT (167492.702 -470291.235),52.795329,2022-01-12 13:30:13,44.451926,61.0,8.343404,True,0.136777
2,2022-01-12 13:32:14,33.768852,-118.193726,2022-01-12 13:32:33,POINT (167350.641 -470208.871),266.678687,2022-01-12 13:31:14,52.795329,60.0,213.883357,True,3.564723


In [22]:
lbt.position_interpolators['9288261']['rt'].progressing_positions >> head(3)

,vehicle_timestamp,vehicle_position_latitude,vehicle_position_longitude,header_timestamp,geometry,shape_meters,last_time,last_loc,secs_from_last,meters_from_last,progressed,speed_from_last
66675,2022-01-12 13:31:14,33.768085,-118.192210,2022-01-12 13:31:33,POINT (167492.702 -470291.235),52.795329,2022-01-12 13:30:13,44.451926,61.0,8.343404,True,0.136777
66676,2022-01-12 13:32:14,33.768852,-118.193726,2022-01-12 13:32:33,POINT (167350.641 -470208.871),266.678687,2022-01-12 13:31:14,52.795329,60.0,213.883357,True,3.564723
66677,2022-01-12 13:33:14,33.770370,-118.193680,2022-01-12 13:33:33,POINT (167351.694 -470040.483),435.011217,2022-01-12 13:32:14,266.678687,60.0,168.332530,True,2.805542


In [3]:
# lbt.position_interpolators['9288261']['rt'].detailed_speed_map()

### Working from an entire Operator x Day

* Includes view for delay at each stop, for all trips with RT data
* Also basic info on how many scheduled trips covered by RT

In [33]:
lbt.stop_delay_view >> head(3)

,stop_id,stop_name,geometry,shape_id,shape_meters,trip_key,trip_id,stop_sequence,arrival_time,route_id,direction_id,actual_time,delay
0,1329,BROADWAY & WISCONSIN SE,POINT (170331.380 -470464.154),1110082,3319.806387,-5721502980648177876,9288975,12,2022-01-12 11:48:34,111,1,2022-01-12 11:48:21.142052,0 days
1,1367,LAKEWOOD & CENTRALIA NE,POINT (171955.580 -462251.798),1110082,17292.340107,-5721502980648177876,9288975,47,2022-01-12 12:28:42,111,1,2022-01-12 12:28:00.969967,0 days
2,1330,BROADWAY & TEMPLE SE,POINT (170602.891 -470529.426),1110082,3599.026438,-5721502980648177876,9288975,13,2022-01-12 11:49:20,111,1,2022-01-12 11:48:57.729418,0 days


In [34]:
(lbt.stop_delay_view
 >> group_by(_.stop_id, _.stop_name)
 >> summarize(mean_delay = _.delay.mean(), max_delay = _.delay.max())
 >> arrange(-_.mean_delay)
 >> head(3)
)

,stop_id,stop_name,mean_delay,max_delay
127,0149,COWLES & SANTA FE SW,0 days 00:33:32.411184,0 days 00:34:21.196251
1502,1996,LOS CERRITOS CENTER LAYOVER,0 days 00:17:58.926773170,0 days 00:38:54.258742
1819,4135,ARTESIA STATION BAY 3,0 days 00:11:05.832202164,0 days 00:30:12.565320


In [35]:
lbt.scheduled_trip_rt_coverage ## Percentage of scheduled trips with at least some RT data

0.9982142857142857

In [36]:
lbt.route_coverage_summary() >> head(3)

,trip_key,trip_id,shape_id,direction_id,min_meters,min_stop,max_meters,max_stop
0,-9217889075259924607,9289433,610106,0,625.200598,3,18562.339122,53
1,-9216178830099537732,9288701,1730122,1,446.220653,2,30983.929803,88
2,-9180052545081174409,9296544,10070,1,2381.776799,4,7748.095459,15


In [23]:
lbt.set_filter(start_time='12:00', end_time='15:00', direction_id='1')

In [24]:
colorscale = branca.colormap.step.RdYlGn_10.scale(vmin=0, 
 vmax=30)
colorscale.caption = "Speed (miles per hour)"

In [26]:
lbt.filter

{'start_time': datetime.time(12, 0),
 'end_time': datetime.time(15, 0),
 'route_ids': None,
 'direction_id': '1',
 'direction': None}

In [44]:
# m = lbt.segment_speed_map(how='low_speeds', colorscale=colorscale)

In [27]:
marin.set_filter(start_time='12:00', end_time='15:00', direction_id='1')

In [43]:
# m2 = marin.segment_speed_map(how='low_speeds', colorscale=colorscale)

In [40]:
marin.position_interpolators['219002']['rt'].progressing_positions >> head(5)

,vehicle_timestamp,vehicle_position_latitude,vehicle_position_longitude,header_timestamp,geometry,shape_meters,last_time,last_loc,secs_from_last,meters_from_last,progressed,speed_from_last
118619,2022-01-12 06:40:03,37.875280,-122.459150,2022-01-12 06:40:07,POINT (-215997.876 -12882.750),375.002366,2022-01-12 06:39:44,345.142822,19.0,29.859544,True,1.571555
118622,2022-01-12 06:40:22,37.874280,-122.460010,2022-01-12 06:40:29,POINT (-216076.280 -12991.919),1303.580789,2022-01-12 06:40:03,375.002366,19.0,928.578424,True,48.872549
118633,2022-01-12 06:42:24,37.874805,-122.459496,2022-01-12 06:42:27,POINT (-216029.630 -12934.748),1377.330697,2022-01-12 06:41:53,1120.231716,31.0,257.098980,True,8.293515
118635,2022-01-12 06:42:45,37.875530,-122.459220,2022-01-12 06:42:47,POINT (-216003.303 -12854.808),1479.794345,2022-01-12 06:42:24,1377.330697,21.0,102.463648,True,4.879221
118638,2022-01-12 06:42:53,37.875620,-122.459390,2022-01-12 06:43:09,POINT (-216017.972 -12844.419),1497.756216,2022-01-12 06:42:45,1479.794345,8.0,17.961871,True,2.245234


In [42]:
# marin.position_interpolators['219002']['rt'].detailed_speed_map()